# Baseball Functions
Most data exported via Statcast Search page on Baseball Savant. (https://baseballsavant.mlb.com/statcast_search) <br>
This is a collection of functions that clean and aggregate pitch-level Statcast data exported from Baseball Savant. <br>
Some of the logic here may be useful in other regards.

In [2]:
import pandas as pd
import numpy as np
import os
import glob
import datetime

df.sv_id.astype('str') + df.pitch_number.astype('str')

len(df.sv_id.dropna().unique())

pitches = df.groupby(level).agg({'sv_id' : 'count'}).reset_index().rename(columns = {'count' : 'pitches'})

bad_bois = df[df.batter.isin(list(pitches[pitches.sv_id == 0].batter.values))]

bad_bois.head(10).iloc[:, 40:50]

# Pitches
df = pd.read_csv('strasburg.csv')
level = 'batter'

pitches = df.groupby(level).agg({level : 'count'}).reset_index().rename(columns = {'count' : 'pitches'})

# Plate Appearances
plate_apps = df[~df.events.replace(np.nan, 'NA').isin(['NA','pickoff_1b'])].groupby(level).agg({level : 'count'})

plate_apps =plate_apps.reset_index().rename(columns = {level : 'plate_apps'})

# At Bats
at_bats = df[~df.events.replace(np.nan, 'NA').isin(['NA'
                                                   ,'pickoff_1b'
                                                   , 'walk'
                                                   , 'intent_walk'
                                                   , 'hit_by_pitch'
                                                  ,'sac_fly'
                                                  ,'sac_bunt'])].groupby(level).agg({level : 'count'})

at_bats = at_bats.reset_index().rename(columns = {level : 'at_bats'})

# Hits
hits = df[df.events.isin(['home_run'
                      ,'single'
                      ,'double'
                      ,'triple'
                     ])].groupby(level).agg({level : 'count'})

hits = hits.reset_index().rename(columns = {level : 'hits'})

# Singles
singles = df[df.events == 'single'].groupby(level).agg({level : 'count'}).reset_index().rename(columns = {level : 'singles'})
# Doubles
doubles = df[df.events == 'double'].groupby(level).agg({level : 'count'}).reset_index().rename(columns = {level : 'doubles'})
# Triples
triples = df[df.events == 'triple'].groupby(level).agg({level : 'count'}).reset_index().rename(columns = {level : 'triples'})
# Home Runs
hrs = df[df.events == 'home_run'].groupby(level).agg({level : 'count'}).reset_index().rename(columns = {level : 'hrs'})
# Walks
walks = df[df.events == 'walk'].groupby(level).agg({level : 'count'}).reset_index().rename(columns = {level : 'walks'})
# Strikeouts
ks = df[df.events.isin(['strikeout','strikeout_double_play'])].groupby(level).agg({level : 'count'}).reset_index().rename(columns = {level : 'strikeouts'})
# Hit By Pitch
hbp = df[df.events == 'hit_by_pitch'].groupby(level).agg({level : 'count'}).reset_index().rename(columns = {level : 'hbp'})
# Weighted Walks
wBB = df[df.events == 'walk'].groupby(level).agg({'wBB' : 'sum'}).reset_index()
# Weighted HBP
wHBP = df[df.events == 'hit_by_pitch'].groupby(level).agg({'wHBP' : 'sum'}).reset_index()
# Weighted Singles
w1B = df[df.events == 'single'].groupby(level).agg({'w1B' : 'sum'}).reset_index()
# Weighted Doubles
w2B = df[df.events == 'double'].groupby(level).agg({'w2B' : 'sum'}).reset_index()
# Weighted Triples
w3B = df[df.events == 'triple'].groupby(level).agg({'w3B' : 'sum'}).reset_index()
# Weighted Home runs
wHR = df[df.events == 'home_run'].groupby(level).agg({'wHR' : 'sum'}).reset_index()

a = pitches.merge(plate_apps, how = 'left', left_on = level, right_on = level)
b = a.merge(at_bats, how = 'left', left_on = level, right_on = level)
c = b.merge(hits, how = 'left', left_on = level, right_on = level)
d = c.merge(singles, how = 'left', left_on = level, right_on = level)
e = d.merge(doubles, how = 'left', left_on = level, right_on = level)
f = e.merge(triples, how = 'left', left_on = level, right_on = level)
g = f.merge(hrs, how = 'left', left_on = level, right_on = level)
h = g.merge(walks, how = 'left', left_on = level, right_on = level)
i = h.merge(hbp, how = 'left', left_on = level, right_on = level)
j = i.merge(wBB, how = 'left', left_on = level, right_on = level)
k = j.merge(wHBP, how = 'left', left_on = level, right_on = level)
l = k.merge(w1B, how = 'left', left_on = level, right_on = level)
m = l.merge(w2B, how = 'left', left_on = level, right_on = level)
n = m.merge(w3B, how = 'left', left_on = level, right_on = level)
o = n.merge(ks, how = 'left', left_on = level, right_on = level)
stats = o.merge(wHR, how = 'left', left_on = level, right_on = level)

stats = stats.replace(np.nan, 0)


# Expected Batting Average

In [10]:
def xbas(row):
    if row['type'] == 'S':
        xba = 0
    else:
        xba = row['estimated_ba_using_speedangle']
    return xba

# Expected WOBA

In [12]:
def xwobas(row):
    if row['type'] == 'B':
        xwoba = row['wBB']
    else:
        xwoba = row.estimated_woba_using_speedangle
    return xwoba

# Last Pitch of At Bat

In [14]:
def last_pitch_abs(df):
    max_pitch_number_per_ab = df.groupby(['game_pk','at_bat_number'], as_index = False).agg({'pitch_number' : 'max'}).rename(columns = {'pitch_number' : 'fp'})
    fp = max_pitch_number_per_ab
    final_pitches = df.merge(fp, left_on = ['game_pk', 'at_bat_number', 'pitch_number'], right_on = ['game_pk', 'at_bat_number', 'fp'])
    final_pitches['xwoba'] = final_pitches.apply(xwobas,axis = 1).fillna(0)
    final_pitches['xba'] = final_pitches.apply(xbas, axis = 1)
    final_pitches = df.merge(final_pitches, left_on = ['game_pk', 'at_bat_number','pitch_number'], right_on = ['game_pk', 'at_bat_number','pitch_number'], how = 'left', suffixes = ('', '_fp'))
    return final_pitches

# Outs

In [16]:
# I could group by inning and inning_topbot
# Loop through each inning-inning_topbot diad, keep a running total of outs_when_up
# Why don't I just identify outs by events?
# No, I would need to subtract out the number of outs

# I think I got it
# "Out" Events feeds a column that gives us the number of outs associated with that events

# Get Stats Function
This function takes in a dataframe (which you should filter) and level to aggregate the data at (ex. player, player/year, team, team/year, etc.) <br>
It will return the result statistics for that

In [18]:
def get_stats(level, df):
    # Pitches

    pitches = df.groupby(level, as_index = False).agg({'description' : 'count'}).rename(columns = {'count' : 'pitches'})

    # Plate Appearances
    plate_apps = df[~df.events.replace(np.nan, 'NA').isin(['NA','pickoff_1b'])].groupby(level,as_index=False).agg({'description' : 'count'})

    plate_apps =plate_apps.rename(columns = {'description' : 'plate_apps'})

    # At Bats
    at_bats = df[~df.events.replace(np.nan, 'NA').isin(['NA'
                                                       ,'pickoff_1b'
                                                       , 'walk'
                                                       , 'intent_walk'
                                                       , 'hit_by_pitch'
                                                      ,'sac_fly'
                                                      ,'sac_bunt'])].groupby(level,as_index=False).agg({'description' : 'count'})

    at_bats = at_bats.rename(columns = {'description' : 'at_bats'})
    
    # Balls in Play
    bip = df[df.type == 'X'].groupby(level,as_index=False).agg({'description' : 'count'}).rename(columns = {'description' : 'bip'})

    # Hits
    hits = df[df.events.isin(['home_run'
                          ,'single'
                          ,'double'
                          ,'triple'
                         ])].groupby(level,as_index=False).agg({'description' : 'count'})

    hits = hits.rename(columns = {'description' : 'hits'})

    # Singles
    singles = df[df.events == 'single'].groupby(level,as_index=True).agg({'description' : 'count'}).rename(columns = {'description' : 'singles'})
    # Doubles
    doubles = df[df.events == 'double'].groupby(level,as_index=True).agg({'description' : 'count'}).rename(columns = {'description' : 'doubles'})
    # Triples
    triples = df[df.events == 'triple'].groupby(level,as_index=True).agg({'description' : 'count'}).rename(columns = {'description' : 'triples'})
    # Home Runs
    hrs = df[df.events == 'home_run'].groupby(level,as_index=True).agg({'description' : 'count'}).rename(columns = {'description' : 'hrs'})
    # Walks
    walks = df[df.events == 'walk'].groupby(level,as_index=True).agg({'description' : 'count'}).rename(columns = {'description' : 'walks'})
    # Strikeouts
    ks = df[df.events.isin(['strikeout','strikeout_double_play'])].groupby(level,as_index=True).agg({'description' : 'count'}).rename(columns = {'description' : 'strikeouts'})
    # Hit By Pitch
    hbp = df[df.events == 'hit_by_pitch'].groupby(level,as_index=True).agg({'description' : 'count'}).rename(columns = {'description' : 'hbp'})
    # Weighted Walks
    wBB = df[df.events == 'walk'].groupby(level,as_index=False).agg({'wBB' : 'sum'})
    # Weighted HBP
    wHBP = df[df.events == 'hit_by_pitch'].groupby(level,as_index=False).agg({'wHBP' : 'sum'})
    # Weighted Singles
    w1B = df[df.events == 'single'].groupby(level,as_index=False).agg({'w1B' : 'sum'})
    # Weighted Doubles
    w2B = df[df.events == 'double'].groupby(level,as_index=False).agg({'w2B' : 'sum'})
    # Weighted Triples
    w3B = df[df.events == 'triple'].groupby(level,as_index=False).agg({'w3B' : 'sum'})
    # Weighted Home runs
    wHR = df[df.events == 'home_run'].groupby(level,as_index=False).agg({'wHR' : 'sum'})
    # xBA
    ###### xba = df.groupby(level,as_index=False).agg({'xba' : 'mean'})
    xba = df.groupby(level,as_index=False).agg({'estimated_ba_using_speedangle' : 'mean'})
    # xWOBA
    ####### xwoba = df.groupby(level,as_index=False).agg({'xwoba' : 'mean'})
    xwoba = df.groupby(level,as_index=False).agg({'estimated_woba_using_speedangle' : 'mean'})
    
    # # Mean
    # mean = 'mean'
    # evMU = df.groupby(level,as_index=False).agg({'launch_speed' : mean
    #                                              ,'launch_angle' : mean
    #                                              ,'hit_distance_sc' : mean
    #                                              ,'release_speed' : mean
    #                                              ,'zone' : mean
    #                                              ,'pitch_number' : mean
    #                                              ,'
    #                                             })
    # # Std
    # std = 'std'
    # for func in funcs:
    #     df.groupby(level,as_index=False).agg({'launch_speed' : func
    #                                              ,'launch_angle' : func
    #                                              ,'hit_distance_sc' : func
    #                                              ,'release_speed' : func
    #                                              ,'zone' : func
    #                                              ,'pitch_number' : funvc
    #                                              ,'
    #                                             })
    # # Max
    # funcs = ['mean', 'std', 'min', 'max', 'var']
                                                 

    a = pitches.merge(plate_apps, how = 'left', left_on = level, right_on = level)
    b = a.merge(at_bats, how = 'left', left_on = level, right_on = level)
    c = b.merge(hits, how = 'left', left_on = level, right_on = level)
    d = c.merge(singles, how = 'left', left_on = level, right_on = level)
    e = d.merge(doubles, how = 'left', left_on = level, right_on = level)
    f = e.merge(triples, how = 'left', left_on = level, right_on = level)
    g = f.merge(hrs, how = 'left', left_on = level, right_on = level)
    h = g.merge(walks, how = 'left', left_on = level, right_on = level)
    i = h.merge(hbp, how = 'left', left_on = level, right_on = level)
    j = i.merge(wBB, how = 'left', left_on = level, right_on = level)
    k = j.merge(wHBP, how = 'left', left_on = level, right_on = level)
    l = k.merge(w1B, how = 'left', left_on = level, right_on = level)
    m = l.merge(w2B, how = 'left', left_on = level, right_on = level)
    n = m.merge(w3B, how = 'left', left_on = level, right_on = level)
    o = n.merge(ks, how = 'left', left_on = level, right_on = level)
    p = o.merge(bip, how = 'left', left_on = level, right_on = level)
    q = p.merge(xba, how = 'left', left_on = level, right_on = level)
    r = q.merge(xwoba, how = 'left', left_on = level, right_on = level)
    stats = r.merge(wHR, how = 'left', left_on = level, right_on = level)

    stats = stats.replace(np.nan, 0)
    
    return stats

## Ex. Batting Performance by Player.
```python
get_stats('player_name', batting)
```

In [20]:
# Extra things to consider
# Additional Columns:
# Expected BA, OBP, SLG, wOBA
# FIP
# Strikeouts
# Innings Pitched
# Runs
# Earned Runs??
# WP


# Measure Calculations
The plan here is to recreate all necessary statistics I can see in real time during a game based solely off their historical playoff data.

In [22]:
def measure_calcs(stats):
    stats.rename(columns = {'batter' : 'pitches'}, inplace = True)

    stats['ba'] = stats.hits/stats.at_bats
    stats['obp'] = (stats.hits + stats.walks + stats.hbp) / (stats.plate_apps)
    stats['slg'] = (stats.singles + 2*stats.doubles + 3*stats.triples + 4*stats.hrs) / (stats.at_bats)
    stats['ops'] = stats.obp + stats.slg
    stats['woba'] = (stats.wBB + stats.wHBP+ stats.w1B+ stats.w2B+ stats.w3B+ stats.wHR) / (stats.plate_apps)
    #stats['xba'] = stats.xba.replacena(0).mean()
    #stats['xwoba'] = stats.xwoba.replacena(0).mean()

    return stats

# Measure Calcs and Get Stats

In [24]:
def mcgs(level,df):
    return measure_calcs(get_stats(level,df)).rename(columns = {'description' : 'pitches'})

# Results - RENAME THIS JAWN KELLEN

In [26]:
def results(level):
    x = mcgs([level], df).loc[:, [##'game_year'
                                                         #'player_name'
                                                         ##,'p_throws'
                                                         ##,'pitch_name'
                                                         ##,'pitch_type'
                                                         
                                                      level   
                                                      ,'pitches'
                                                      ,'plate_apps'
                                                      ##,'at_bats'
                                                      ,'bip'
                                                      ,'hits'
                                                      ##,'singles'
                                                      ##,'doubles'
                                                      ##,'triples'
                                                      ,'hrs'
                                                      ,'walks'
                                                      ,'strikeouts'
                                                      ##,'hbp'
                                                      ##,'wBB'
                                                      ##,'wHBP'
                                                      ##,'w1B'
                                                      ##,'w2B'
                                                      ##,'w3B'
                                                      ##,'wHR'
                                                      ,'ba'
                                                      ##,'xba'
                                                      ,'obp'
                                                      ,'slg'
                                                      ,'ops'
                                                      ,'woba'
                                                      ##,'xwoba'
                                                     ]
                                                 ].round(3)
    return x

# Group By

In [28]:
def gb(df,**args): ## stat,calc,level,team, criteria
    df = last_pitch_abs(df)
    df = df[criteria]
    po = df[((df.home_team == team)
                 &(df.inning_topbot == 'Bot')
                  )
                 |
                 ((df.away_team == team)
                  &(df.inning_topbot == 'Top')
                 )]

    pp = df[((df.home_team == team)
             &(df.inning_topbot == 'Top')
            )
            |
            ((df.away_team == team)
             &(df.inning_topbot == 'Bot')
            )]

    final = df.groupby(level,as_index=False).agg({stat : calc}).reset_index(drop=False
                                                              ).reset_index(drop=False
                                                                           ).rename(columns = {'level_0' : 'rank'
                                                                                               ,stat : '{}_{}'.format(calc,stat)
                                                                                              })
    return final

# Can I do the same function using apply or transform?
This would give me more flexibility to write my own functions to apply to and/or transform my grouped data

# Compare

In [31]:
def compare(df):
    return df

# Graph

In [33]:
def graph(x,y, title, xlabel, ylabel, colors):
    r = round(x.corr(y),2) # Calculate correlation coefficient
    m, b = np.polyfit(x, y, 1) # Fit a polynomial between the two variables
    #title = 'Velo vs. Spin Rate' # Title the graph
    #xlabel = 'Spin' # Label the x axis
    #ylabel = 'Velo' # Label the y axis

    # Build the plot
    plt.scatter(x, y, c = colors, alpha = 1) # Scatter plot, intentionally made transparent
    plt.plot(x.values, (m*x + b).values, color = '#003469') # Regression line
    plt.axvline(x = x.mean(), color = '#000000') # Vertical line at median of x
    plt.axhline(y = y.mean(), color = '#000000') # Horizontal Line at median of y
    plt.suptitle(title, fontsize = 14, fontname = 'Verdana') # Title
    plt.title('n = {}. r = {}. Avg {} = {}, +-{}. Avg {} = {}, +-{}'.format(len(x)
                                                                    ,r
                                                                      ,xlabel
                                                                      ,round(x.mean(),1)
                                                                      ,round(x.std(),1)
                                                                      ,ylabel
                                                                      ,round(y.mean(),1)
                                                                      ,round(y.std(),1)
                                                                     ), fontsize = 8, fontname = 'Verdana') # Subtitle
    plt.xlabel(xlabel) # Label the x axis
    plt.ylabel(ylabel) # Label the y axis
    plt.show()

# Month Trend

In [35]:
# Time Trend

In [36]:
def pitch_mix(df):
    df = df.groupby(['pitch_type', 'pitch_name'],as_index=False).agg({'description' : 'count'
                                                                      ,'release_speed' : 'mean'
                                                                      ,'release_spin_rate' : 'mean'}
                                                                    ).round(1
                                                                           ).rename(columns = {'description' : 'pitches'})
    return df

# Get Non-Phillies Player Data

In [38]:
def get_nphillies_data():
    #%run "Baseball Functions.ipynb"
    woba_weights = pd.read_csv('wOBA and FIP Constants.csv')

    # Load All Non-Phillies Player Data
    np_files = [i for i in glob.glob('*_np.csv')]
    nphl = pd.DataFrame()
    for f in np_files:
        df = pd.read_csv(f)
        nphl = pd.concat([nphl,df])

    nphl['source'] = 'n'
    nphl = nphl.merge(woba_weights, left_on = 'game_year', right_on = 'Season', suffixes = ('','_y'))

    print('Length of nphl dataframe: ' + str(len(nphl)))    
    return nphl

## Code Example
```python
phils_sc, pos, pps = get_phillies_data()
nphl = get_nphillies_data()
```

# Get Phillies Spring Training Data

In [41]:
def get_phillies_spring_training_data():
    ppst24 = pd.read_csv('spring_training_2024 ppst.csv')
    post24 = pd.read_csv('spring_training_2024 post.csv')
    phils_st_2024 = pd.concat([pps, pos])
    return phils_st_2024, ppst24, post24

# Get Phillies Data

In [43]:
def get_phillies_data():
    os.chdir('C:\\Users\\Kellen\\OneDrive\\Documents\\Python Scripts\\MLB')
    reg_po23 = pd.read_csv('reg_po23.csv')
    reg_pp23 = pd.read_csv('reg_pp23.csv')
    post_po23 = pd.read_csv('post_po23.csv')
    post_pp23 = pd.read_csv('post_pp23.csv')

    po23 = pd.concat([reg_po23, post_po23])
    pp23 = pd.concat([reg_pp23, post_pp23])

    woba_weights = pd.read_csv('wOBA and FIP Constants.csv')
    
    # Load All Phillies Player Data
    # Because of the additional name with reg and post
    pos = pd.DataFrame()
    pps = pd.DataFrame()
    phils_sc = pd.DataFrame()

    regpo22 = pd.read_csv('reg_po22.csv')
    regpp22 = pd.read_csv('reg_pp22.csv')
    postpo22 = pd.read_csv('post_po22.csv')
    postpp22 = pd.read_csv('post_pp22.csv')

    po22 = pd.concat([regpo22, postpo22])
    pp22 = pd.concat([regpp22, postpp22])

    for k in ['15','16','17','18','19','20','21']:
        po = [i for i in glob.glob('*po{}.csv'.format(str(k)))]
        dfpo = pd.read_csv(po[0])
        pos = pd.concat([pos,dfpo])

        pp = [i for i in glob.glob('*pp{}.csv'.format(str(k)))]
        dfpp = pd.read_csv(pp[0])
        pps = pd.concat([pps, dfpp])

        
    #pst24 = ppst24 = post24 = get_phillies_spring_training_data()
    post24 = po24 = pd.read_csv('po24.csv')#DataFrame()
    ppst24 = pp24 = pd.read_csv('pp24.csv')#DataFrame()
    pos = pd.concat([pos, po22, po23, post24]).merge(woba_weights, left_on = 'game_year', right_on = 'Season', suffixes = ('_bad',''))
    pps = pd.concat([pps, pp22, pp23, ppst24]).merge(woba_weights, left_on = 'game_year', right_on = 'Season', suffixes = ('_bad',''))    
    #phils_st_2024 = pd.concat([pps, pos]).merge(woba_weights, left_on = 'game_year', right_on = 'Season', suffixes = ('_bad', ''))
    phils_sc = pd.concat([pos,pps]).merge(woba_weights, left_on = 'game_year', right_on = 'Season', suffixes = ('','_yamomma'))
    

    print('Length of phils_sc dataframe: ' + str(len(phils_sc))) # Should it be 407,099? I think so.
    return phils_sc, pos, pps

In [44]:
# Update MLB Data

# Get Pitcher Data

In [46]:
def get_pitcher_data(df,pitcher_name):
    player_id = df[df.player_name == pitcher_name].pitcher.unique()[0]
    player_df = df[df.pitcher == player_id]
    return player_df

# Get Batter Data

In [48]:
def get_batter_data(df,batter_name):
    player_id = df[df.player_name == batter_name].batter.unique()[0]
    player_df = df[df.batter == player_id]
    return player_df

## Pitch Types

## Front Hip Sinker

In [51]:
# def front_hip_sinker(df):
#     lefties = df[(df.zone.isin([3,6,9]))
#                  &(df.pitch_type == 'SI')
#                  #&(df.pitch_location == )
#                  &(df.stand == 'L')
#                 ]
#     righties = df[(df.zone.isin([1,4,7]))
#                    &(df.pitch_type == 'SI')
#                    &(df.stand == 'R')
#                  ]
#     fhs = pd.concat([lefties, righties])
#     return fhs

In [52]:
# def front_hip_sinker(df):
#     lhb = lefties = df[(df.zone.isin([3,6,9]))
#                        &(df.pitch_type == 'SI')
#                       #&(df.pitch_location == )
#                        &(df.stand == 'L')
#                       ]
#     rhb = righties = df[(df.zone.isin([1,4,7]))
#                    &(df.pitch_type == 'SI')
#                    &(df.stand == 'R')
#                  ]
#     fhs = pd.concat([lefties, righties])
#     return fhs

# df = front_hip_sinker(nola)
# df[df.stand == 'L'].zone.unique()

In [53]:
def lhb_zones():
    up_away = zha = l_ha = 1 
    up = zhc = l_ha = 2
    up_in = zhi = l_ha = 3
    middle_away = zma = l_ha = 4
    middle_middle = zmc = l_ha = 5
    middle_in = zmi = 6
    low_away = zla = l_ha = 7 
    down = zlc = l_ha = 8 
    down_in = zli = l_ha = 9

# lhb_zones = [zha,zhc,zhi ## 1, 2, 3
#             ,zma,zmc,zmi ## 4, 5, 6
#             ,zla,zlc,zli ## 7, 8, 9
#             ]

# (TO DO) Get Fielder Data

# Merge Curveballs

In [56]:
def merge_cu(df):
    df['pitch_name'] = np.where(df['pitch_type'].isin(['KC', 'CU']), 'Curveball', df['pitch_name']) # Thanks to ChatGPT
    df['pitch_type'] = np.where(df['pitch_type'].isin(['KC', 'CU']), 'CU', df['pitch_type'])
    return df

#df_cu = merge_cu(df)

# Pitch Mix

In [58]:
def pitch_mix(df):
    # Pitch Mix
    pm  = df.groupby(['pitch_type', 'pitch_name'], as_index=False).agg({'des' : 'count'
                                                                        ,'release_speed' : 'mean'
                                                                        ,'release_spin_rate' : 'mean'
                                                                        ,'zone' : 'mean'
                                                                       }
                                                                    ).rename(columns = {'des' : 'count'}
                                                                            ).sort_values(by = 'count', ascending = False)
    pm['usage'] = round((pm['count']/len(df))*100,1)
    return pm

# Pitch Mix By Group

In [60]:
def pitch_mix_by_group(df, group):
    # Pitch Mix
    pm  = df.groupby(group, as_index=False).agg({'des' : 'count'
                                                   ,'release_speed' : 'mean'
                                                   ,'release_spin_rate' : 'mean'
                                                   ,'zone' : 'mean'
                                                  }
                                                 ).rename(columns = {'des' : 'count'}
                                                         ).sort_values(by = 'count', ascending = False).round(3)
    pm['usage'] = round((pm['count']/len(df))*100,1)
    return pm

In [61]:
# def pitch_mix_by_group(df, **kwargs):
#     df[
#     # Pitch Mix
#     pm  = df.groupby([group], as_index=False).agg({'des' : 'count'
#                                                    ,'release_speed' : 'mean'
#                                                    ,'release_spin_rate' : 'mean'
#                                                   }
#                                                  ).rename(columns = {'des' : 'count'}
#                                                          ).sort_values(by = 'count', ascending = False).round(3)
#     pm['usage'] = round((pm['count']/len(df))*100,1)
#     return pm
# pitch_mix_by_group(nola, ['stand','pitch_type'])

# Filtered Dataframe produced by ChatGPT

In [63]:
import pandas as pd

def filter_dataframe(dataframe, **kwargs):
    """
    Filter a DataFrame based on key-value pairs in kwargs.

    Parameters:
    - dataframe: pandas DataFrame
    - **kwargs: Key-value pairs where keys are column names and values are the filter criteria.

    Returns:
    - pandas DataFrame: Filtered DataFrame
    """

    # Ensure that the input is a pandas DataFrame
    if not isinstance(dataframe, pd.DataFrame):
        raise ValueError("Input must be a pandas DataFrame")

    # Initialize a boolean mask with True values for all rows
    mask = pd.Series(True, index=dataframe.index)

    # Apply filters based on key-value pairs in kwargs
    for column, value in kwargs.items():
        if column not in dataframe.columns:
            raise ValueError(f"Column '{column}' does not exist in the DataFrame")
        
        mask = mask & (dataframe[column] == value)

    # Apply the boolean mask to filter the DataFrame
    filtered_dataframe = dataframe[mask]

    return filtered_dataframe

# Example usage:
# Assuming you have a DataFrame called 'your_dataframe'
# filtered_result = filter_dataframe(your_dataframe, column1=value1, column2=value2, ...)


In [64]:
def lhb_pitch_mix(df):
    return pitch_mix(df[df.stand == 'L'])

In [65]:
def rhb_pitch_mix(df):
    return pitch_mix(df[df.stand == 'R'])

# Pitch Mix by Season

In [67]:
def pitch_mix_by_season(df):
    pitch_mix_by_season = pd.DataFrame()
    for y in df.game_year.unique().tolist():
        pmbs = pitch_mix(df[(df.game_year == y)
                            #&(df.stand == s)
                           ]
                        )
        pmbs['year'] = y
        pitch_mix_by_season = pd.concat([pitch_mix_by_season, pmbs])
            
    return pitch_mix_by_season
#pitch_mix_by_season(df)

# Pitch Mix by Season and Stand

In [69]:
def pitch_mix_by_season_and_stand(df):
    pitch_mix_by_season = pd.DataFrame()
    for y in df.game_year.unique().tolist():
        for s in df.stand.unique().tolist():
            pmbs = pitch_mix(df[(df.game_year == y)
                                &(df.stand == s)]
                            )
            pmbs['year'] = y
            pmbs['stand'] = s
            pitch_mix_by_season = pd.concat([pitch_mix_by_season, pmbs])
            
    return pitch_mix_by_season
#pitch_mix_by_season(df)

# KS Filter Function

**Use Case**: "I want to see pitch level data of Bryce Harper's 2020 season at the plate."

```python
# My Filtering Function Example
df = phils_sc
field = 'game_year'
criteria = 2020

df = ks_filter(pd.concat([phils_sc,nphl]),'player_name','Harper, Bryce')
```

In [73]:
def ks_filter(df,field,criteria):
    df = df[df[field] == criteria]
    return df

# Summarize Events

**Use Case**: "I want to see how many strikeouts Kyle Schwarber has in his career"

```python
df = pos
level = 'game_year'

tgt = ks_filter(df, 'player_name', 'Schwarber, Kyle')

## Single Event
e = event
summarize_events(tgt, e)

## Loop Through All Events
for e in tgt.events.unique().tolist():
    summarize_events(tgt, e)
```

In [76]:
def summarize_events(df,event):
    print('Pitches: ', len(df)
          ,'| Games: ', df.game_pk.nunique()
          ,'| {}: '.format(event), len(df[df.events == event])
     )

# Column Minimum

In [78]:
def col_min(df, f, crit, c):
    df = df[df[f] == crit]
    df = df[df[c] == df[c].min()]
    return df.loc[:, ['game_date'
                      ,'player_name'
                      ,c
                      ,'inning'
                      ,'estimated_ba_using_speedangle'
                     ]
                 ]

# Column Maximum

In [80]:
def col_max(df, f, crit, c):
    df = df[df[f] == crit]
    df = df[df[c] == df[c].max()]
    return df.loc[:, ['game_date'
                      ,'player_name'
                      ,c
                      ,'inning'
                      ,'estimated_ba_using_speedangle'
                     ]
                 ]

# Column Average

In [82]:
def col_mean(df, f, crit, c):
    df = df[df[f] == crit]
    df = df[df[c] == df[c].mean()]
    return df.loc[:, ['game_date'
                      ,'player_name'
                      ,c
                      ,'inning'
                      ,'estimated_ba_using_speedangle'
                     ]
                 ]

# Column Standard Deviation

In [84]:
def col_std(df, f, crit, c):
    df = df[df[f] == crit]
    df = df[df[c] == df[c].std()]
    return df.loc[:, ['game_date'
                      ,'player_name'
                      ,c
                      ,'inning'
                      ,'estimated_ba_using_speedangle'
                     ]
                 ]

# Column Median

In [86]:
def col_median(df, f, crit, c):
    df = df[df[f] == crit]
    df = df[df[c] == df[c].median()]
    return df.loc[:, ['game_date'
                      ,'player_name'
                      ,c
                      ,'inning'
                      ,'estimated_ba_using_speedangle'
                     ]
                 ]

# Battling the tism Allegations by Listing Off the Phillies Rosters, player_names, player_ids
Except I am not battling any tism at all here, I am deriving a way to get that information without just having to list off everyone. But I will have to add a 'jersey' column to the rosters.

phils_sc, pos, pps = get_phillies_data()

pps.groupby(['player_name'
             ,'game_year'
            ],as_index=False
           ).agg({'pitcher' : 'nunique'})